# Real-time EEG Mental State Inference

This notebook demonstrates how to use the EEG mental state inference system in real-time.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime

# Import our modules
from common.mental_state_inference import (
    generate_mental_state_mapping,
    EEGRealTimePredictor,
    real_time_eeg_demo
)

## 1. Load Models and Generate Mental State Mapping

In [ ]:
# Set up paths
model_dir = './models/clustering'
data_path = '../data'  # Change to your data directory

# Generate mapping from clusters to mental states
mental_state_mapping = generate_mental_state_mapping(
    model_dir=model_dir,
    data_sample_path=data_path,
    method='feature_based'
)

print(f"\nMental state mapping: {mental_state_mapping}")

## 2. Initialize Real-time Predictor

In [ ]:
# Initialize real-time predictor
predictor = EEGRealTimePredictor(
    model_path=model_dir,
    buffer_size=30,  # Number of samples to collect before prediction
    step_size=5,     # Step size for sliding window
    mental_state_mapping=mental_state_mapping
)

## 3. Demo with Pre-recorded Data

This demonstrates how the real-time prediction works using pre-recorded data.

In [ ]:
# Run real-time demo with pre-recorded data
results = real_time_eeg_demo(
    predictor=predictor,
    data_path=data_path,
    duration_seconds=30,  # Run for 30 seconds
    sample_interval=0.1   # 10Hz sampling rate
)

## 4. Interactive Widget Demo

This shows how you might integrate the predictor with interactive widgets.

In [ ]:
# Set up widgets
output = widgets.Output()
progress = widgets.FloatProgress(value=0, min=0, max=1, description='Buffer:')
state_label = widgets.Label(value='Mental State: Waiting for data')
confidence_label = widgets.Label(value='Confidence: -')
start_button = widgets.Button(description='Start Demo')
stop_button = widgets.Button(description='Stop')

# Display the widgets
display(widgets.HBox([start_button, stop_button]))
display(progress, state_label, confidence_label)
display(output)

# Variables to control the demo
demo_running = False

# Function to run when Start button is clicked
def on_start_button_clicked(b):
    global demo_running
    demo_running = True
    
    # Clear the predictor's buffer
    predictor.clear_buffer()
    
    # Start the demo in a new thread
    import threading
    demo_thread = threading.Thread(target=run_interactive_demo)
    demo_thread.daemon = True
    demo_thread.start()

# Function to run when Stop button is clicked
def on_stop_button_clicked(b):
    global demo_running
    demo_running = False

# Connect the button click events
start_button.on_click(on_start_button_clicked)
stop_button.on_click(on_stop_button_clicked)

# Function to run the interactive demo
def run_interactive_demo():
    global demo_running
    
    # Load sample data
    from common.helpers import load_eeg_data
    if os.path.isdir(data_path):
        file_dfs, _ = load_eeg_data(directory_path=data_path)
        # Use first file
        sample_df = list(file_dfs.values())[0]
    else:
        _, sample_df = load_eeg_data(single_file=data_path)
    
    # Process samples
    sample_index = 0
    while demo_running and sample_index < len(sample_df):
        # Get sample
        sample = sample_df.iloc[sample_index]
        
        # Add to predictor
        buffer_full = predictor.add_sample(sample)
        
        # Update progress
        progress.value = len(predictor.buffer) / predictor.buffer_size
        
        # Make prediction if buffer is full
        if buffer_full:
            prediction = predictor.predict()
            
            # Update widgets
            if prediction['state']:
                state_color = 'green' if prediction['state'] == 'focused' else 'blue'
                state_label.value = f"Mental State: <span style='color:{state_color}'>{prediction['state'].upper()}</span>"
            else:
                state_label.value = "Mental State: Waiting for data"
                
            confidence_label.value = f"Confidence: {prediction['confidence']:.2f}"
            
            # Add to output
            with output:
                clear_output(wait=True)
                print(f"Time: {datetime.now().strftime('%H:%M:%S.%f')[:-3]}")
                print(f"State: {prediction['state']}")
                print(f"Cluster: {prediction['cluster']}")
                print(f"Confidence: {prediction['confidence']:.2f}")
                print(f"Prediction time: {prediction.get('prediction_time', 0)*1000:.1f} ms")
                
                # Plot the most recent prediction history
                if len(predictor.recent_predictions) > 1:
                    plt.figure(figsize=(8, 2))
                    state_nums = [1 if s == 'focused' else 0 for s in predictor.recent_predictions]
                    plt.plot(state_nums, 'o-')
                    plt.yticks([0, 1], ['Relaxed', 'Focused'])
                    plt.title('Recent Predictions')
                    plt.tight_layout()
                    plt.show()
        
        # Move to next sample
        sample_index += 1
        
        # Sleep to simulate real-time
        time.sleep(0.1)
    
    # Update status when done
    if not demo_running:
        with output:
            print("Demo stopped by user")
    else:
        with output:
            print("Demo completed - reached end of data")
            
    demo_running = False

## 5. Manual Testing

You can also manually test the predictor with your own data.

In [ ]:
# Create a sample EEG datapoint
# This would typically come from your EEG device
# The structure should match your training data
sample_data = {
    'Delta': 0.5,
    'Theta': 0.3,
    'Alpha': 0.8,
    'Beta': 0.2,
    'Gamma': 0.1,
    # Add other channels/features as needed
}

# Reset the predictor
predictor.clear_buffer()

# Add the sample
predictor.add_sample(sample_data)

# Check buffer status
print(f"Buffer fill level: {len(predictor.buffer)}/{predictor.buffer_size}")

# Note: You would need to add more samples to fill the buffer before making a prediction
# In a real application, you would add samples as they come in from your EEG device

## 6. Integration with Real EEG Device

This is a template for how you might integrate with a real EEG device:

In [ ]:
def integrate_with_eeg_device():
    """
    Template function for integrating with a real EEG device
    Replace the comments with actual device-specific code
    """
    # Initialize the predictor
    predictor = EEGRealTimePredictor(
        model_path=model_dir,
        buffer_size=30,
        step_size=5,
        mental_state_mapping=mental_state_mapping
    )
    
    # Initialize your EEG device
    # eeg_device = YourEEGDevice()
    # eeg_device.connect()
    
    try:
        # Main loop
        while True:
            # Get data from your EEG device
            # raw_eeg_data = eeg_device.get_data()
            
            # Process the raw data into the format expected by the predictor
            # processed_data = process_raw_eeg(raw_eeg_data)
            
            # Add to the predictor
            # predictor.add_sample(processed_data)
            
            # Make prediction if buffer is full
            # if len(predictor.buffer) >= predictor.buffer_size:
            #     prediction = predictor.predict()
            #     print(f"Mental state: {prediction['state']}")
            
            # Sleep to control sampling rate
            time.sleep(0.1)  # 10Hz sampling rate
            
    except KeyboardInterrupt:
        print("Stopping...")
    finally:
        # Clean up
        # eeg_device.disconnect()
        pass

# Uncomment to run with a real device
# integrate_with_eeg_device()